In [14]:

import pandas as pd
from selenium import webdriver
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import re
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException, NoSuchWindowException

from selenium.webdriver.common.by import By

In [15]:
chrome_options = webdriver.ChromeOptions()
prefs = {"profile.default_content_setting_values.notifications": 2}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
driver.get('https://www.imdb.com/search/title/?moviemeter=1501,2000')

In [16]:
# Hàm lọc
def filter_array(array_elements):
    array = []
    for element in array_elements:
        array_processed = element.text.replace('\n', '').strip()
        array.append(array_processed)
    return array

# Đợi cho phần tử "See more" hiển thị
wait = WebDriverWait(driver, 10)
see_more_button = wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'ipc-see-more__button')]")))

#Cuộn trang web xuống tới cuối
while see_more_button.is_displayed():
    actions = ActionChains(driver)
    actions.move_to_element(see_more_button).perform()
    try:
        driver.execute_script("arguments[0].click();", see_more_button)
        see_more_button = wait.until(EC.visibility_of_element_located((By.XPATH, "//button[contains(@class, 'ipc-see-more__button')]")))
    except:
        break


In [17]:
# Lọc description
description = []
description_element = None
try:
    description_element = driver.find_element(By.CSS_SELECTOR, 'div.ipc-html-content.ipc-html-content--base.sc-74bf520e-0.iiRKKi.dli-plot-container div.ipc-html-content-inner-div')
except NoSuchElementException:
    pass

description_text = description_element.text if description_element else None
description.append(description_text)
# Lọc movie link
movie_link_elements = driver.find_elements(By.CSS_SELECTOR,'a.ipc-title-link-wrapper')
movie_links = [element.get_attribute('href') for element in movie_link_elements]

movies = []
release_date = []
film_actors = []
duration = []
country = []
companies = []
movie_category = []
ratings = []
directors = []
Certificate = []
budget= []
boxoffice= []
original_language = []
taglines = []
writing_credits = []

In [18]:

for movie_link in movie_links:
    driver.get(movie_link)
    # Lọc movie
    try:
        movie = driver.find_element('css selector', 'h1[data-testid="hero__pageTitle"] span.hero__primary-text')
        if movie:
            movies.append(movie.text)  
        else:
            movies.append(None)  
    except:
        movies.append(None)  
    # Lọc diễn viên
    actor_elements = driver.find_elements(By.CSS_SELECTOR,'a[data-testid="title-cast-item__actor"]')
    actor_text = ', '.join([element.text for element in actor_elements])
    film_actors.append(actor_text)
    # Lọc duration
    duration_element = None
    try:
        duration_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-techspec_runtime"] div.ipc-metadata-list-item__content-container')
    except NoSuchElementException:
        pass
    duration_text = duration_element.text if duration_element else None
    duration.append(duration_text)
    #Lọc ratings
    rating_elements = None
    try:
        rating_elements = driver.find_element(By.CSS_SELECTOR,'span.sc-bde20123-1.cMEQkK')
    except NoSuchElementException:
        pass
    rating_text = rating_elements.text if rating_elements else None
    ratings.append(rating_text)
    # Lọc release date
    release_date_element = None
    try:
        release_date_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-details-releasedate"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item a.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    except NoSuchElementException:
        pass
    release_date_text = release_date_element.text if release_date_element else None
    release_date.append(release_date_text)
    # Lọc country
    country_element = None
    try:
        country_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-details-origin"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item a.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    except NoSuchElementException:
        pass
    country_text = country_element.text if country_element else None
    country.append(country_text)
    # Lọc budget và box office
    budget_element = None
    boxoffice_element = None
    try:
        budget_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-boxoffice-budget"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item span.ipc-metadata-list-item__list-content-item')
        boxoffice_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-boxoffice-cumulativeworldwidegross"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item span.ipc-metadata-list-item__list-content-item')
    except NoSuchElementException:
        pass
    budget_text = budget_element.text if budget_element else None
    budget.append(budget_text)
    boxoffice_text = boxoffice_element.text if boxoffice_element else None
    boxoffice.append(boxoffice_text)
    #Lọc Certificate
    certificate_element = None
    try:
        certificate_element = driver.find_element(By.XPATH, '//a[contains(@href, "/parentalguide/certificates")]')
    except NoSuchElementException:
        pass

    certificate_text = certificate_element.text if certificate_element else None
    Certificate.append(certificate_text)

    # Lọc original_language
    original_language_element = None
    try:
        original_language_element = driver.find_element(By.CSS_SELECTOR,'li[data-testid="title-details-languages"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li.ipc-inline-list__item a.ipc-metadata-list-item__list-content-item.ipc-metadata-list-item__list-content-item--link')
    except NoSuchElementException:
        pass
    
    original_language_text = original_language_element.text if original_language_element else None
    original_language.append(original_language_text)
    # Lọc companies
    companies_elements = None
    try:
        companies_elements = driver.find_elements(By.CSS_SELECTOR,'li[data-testid="title-details-companies"] div.ipc-metadata-list-item__content-container ul.ipc-inline-list.ipc-inline-list--show-dividers.ipc-inline-list--inline.ipc-metadata-list-item__list-content.base li')
    except NoSuchElementException:
        pass

    companies_text = ''
    for i, element in enumerate(companies_elements):
        if i < len(companies_elements) - 1:
            companies_text += element.text + ', '
        else:
            companies_text += element.text

    companies.append(companies_text)
    # Lọc genre
    movie_category_elements = None
    movie_category_elements = driver.find_elements(By.CSS_SELECTOR,'a.ipc-chip.ipc-chip--on-baseAlt span.ipc-chip__text')
    movie_category_text = ', '.join([element.text for element in movie_category_elements]) if movie_category_elements else None
    movie_category.append(movie_category_text)
    #print(movie_category)
    #Lọc director
    try:
        full_cast_crew_link = driver.find_element(By.CSS_SELECTOR, 'a[aria-label="See full cast and crew"]')
        driver.execute_script("arguments[0].click();", full_cast_crew_link)
        time.sleep(3)  # wait for the page to load
        
        director_tag_element = driver.find_element(By.CSS_SELECTOR, 'h4[id="director"]')
        table_element = director_tag_element.find_element(By.XPATH, './following-sibling::table[@class="simpleTable simpleCreditsTable"]')
        director_elements = table_element.find_elements(By.CSS_SELECTOR, 'td.name a[href^="/name/"]')
        directors_text = ', '.join([element.text for element in director_elements]) if director_elements else None
        directors.append(directors_text)
    except NoSuchElementException:
        directors.append(None)
    # Lọc Writing Credits
    writing_credits_tag_element = None
    try:
        writing_credits_tag_element = driver.find_element(By.CSS_SELECTOR,'h4[id="writer"]')
    except NoSuchElementException:
        pass

    if writing_credits_tag_element:
        table_element = writing_credits_tag_element.find_element(By.XPATH,'./following-sibling::table[@class="simpleTable simpleCreditsTable"]')
        writing_credits_element = table_element.find_elements(By.CSS_SELECTOR,'td.name a[href^="/name/"]')
    else:
        writing_credits_element = None

    writing_credits_text = ', '.join([element.text for element in writing_credits_element]) if writing_credits_element else None
    writing_credits.append(writing_credits_text)


In [19]:
# Define the expected length
expected_length = len(movie_links)

# Function to pad lists with None
def pad_list(lst, length):
    return lst + [None] * (length - len(lst))

# Pad all lists to the same length
movies = pad_list(movies, expected_length)
film_actors = pad_list(film_actors, expected_length)
duration = pad_list(duration, expected_length)
ratings = pad_list(ratings, expected_length)
release_date = pad_list(release_date, expected_length)
country = pad_list(country, expected_length)
budget = pad_list(budget, expected_length)
boxoffice = pad_list(boxoffice, expected_length)
certificate = pad_list(Certificate, expected_length)
original_language = pad_list(original_language, expected_length)
companies = pad_list(companies, expected_length)
movie_category = pad_list(movie_category, expected_length)
directors = pad_list(directors, expected_length)
writing_credits = pad_list(writing_credits, expected_length)


In [20]:
import pandas as pd

# Create a dictionary of the lists
data = {
    'Links' : movie_links,
    'Movie': movies,
    'Actors': film_actors,
    'Duration': duration,
    'Ratings': ratings,
    'Release Date': release_date,
    'Country': country,
    'Budget': budget,
    'Box Office': boxoffice,
    'Certificate': certificate,
    'Original Language': original_language,
    'Companies': companies,
    'Genres': movie_category,
    'Directors': directors,
    'Writing Credits': writing_credits
}

# Create the DataFrame
df = pd.DataFrame(data)

# Output the DataFrame to check

# Export the DataFrame to a CSV file
df.to_csv('movies_data.csv', index=False)

print("DataFrame successfully exported to movies_data.csv")


DataFrame successfully exported to movies_data.csv


In [21]:
df

,Links,Movie,Actors,Duration,Ratings,Release Date,Country,Budget,Box Office,Certificate,Original Language,Companies,Genres,Directors,Writing Credits
0,https://www.imdb.com/title/tt1355644/?ref_=sr_t_1,Người Du Hành,"Jennifer Lawrence, Chris Pratt, Michael Sheen,...",1 hour 56 minutes,None,"December 21, 2016 (Vietnam)",United States,"$110,000,000 (estimated)","$303,144,152",C16,English,"Columbia Pictures, LStar Capital, Village Road...","Space Sci-Fi, Drama, Romance, Sci-Fi, Thriller",Morten Tyldum,Jon Spaihts
1,https://www.imdb.com/title/tt1311067/?ref_=sr_t_2,Lễ Hội Kinh Hoàng 2,"Scout Taylor-Compton, Tyler Mane, Malcolm McDo...",1 hour 45 minutes,None,"August 28, 2009 (United States)",United States,"$15,000,000 (estimated)","$39,421,467",R,English,"Dimension Films, Spectacle Entertainment Group...","Slasher Horror, Horror",Rob Zombie,Rob Zombie
2,https://www.imdb.com/title/tt33060158/?ref_=sr...,Autumn at Apple Hill,"Erin Cahill, Wes Brown, Paula Boudreau, Sarah ...",1 hour 24 minutes,None,"October 5, 2024 (United States)",United States,None,None,TV-G,English,"Cartel Pictures, Crown Media Productions, Hall...","Romantic Comedy, Comedy, Romance",Séan Geraughty,"Paul Ditty, Angie Ellington"
3,https://www.imdb.com/title/tt28277817/?ref_=sr...,Bird,"Nykiya Adams, Franz Rogowski, Barry Keoghan, J...",1 hour 59 minutes,None,"November 8, 2024 (United States)",United Kingdom,None,None,R,English,"BBC Film, British Film Institute (BFI), Pinky ...",Drama,Andrea Arnold,Andrea Arnold
4,https://www.imdb.com/title/tt1655441/?ref_=sr_t_5,Sắc Đẹp Vĩnh Cửu,"Blake Lively, Michiel Huisman, Harrison Ford, ...",1 hour 52 minutes,None,"June 26, 2015 (Vietnam)",United States,"$25,000,000 (estimated)","$65,663,276",P,English,"Lakeshore Entertainment, Lionsgate, Sidney Kim...","Feel-Good Romance, Supernatural Fantasy, Drama...",Lee Toland Krieger,"J. Mills Goodloe, Salvador Paskowitz, J. Mills..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,https://www.imdb.com/title/tt0955308/?ref_=sr_...,Huyền Thoại Robin Hood,"Russell Crowe, Cate Blanchett, Matthew Macfady...",2 hours 20 minutes,None,"May 21, 2010 (Vietnam)",United States,"$200,000,000 (estimated)","$321,669,741",PG-13,English,"Universal Pictures, Imagine Entertainment, Rel...","Action Epic, Epic, Period Drama, Swashbuckler,...",Ridley Scott,"Brian Helgeland, Brian Helgeland, Ethan Reiff,..."
496,https://www.imdb.com/title/tt1323594/?ref_=sr_...,Kẻ Trộm Mặt Trăng,"Steve Carell, Jason Segel, Russell Brand, Juli...",1 hour 35 minutes,None,"July 20, 2010 (Vietnam)",United States,"$69,000,000 (estimated)","$544,174,973",PG,English,"Universal Pictures, Illumination Entertainment","Computer Animation, Slapstick, Urban Adventure...","Pierre Coffin, Chris Renaud","Cinco Paul, Ken Daurio, Sergio Pablos"
497,https://www.imdb.com/title/tt0108037/?ref_=sr_...,Lũ Quỷ Nhỏ,"Tom Guiry, Mike Vitar, Art LaFleur, Patrick Re...",1 hour 41 minutes,None,"April 7, 1993 (United States)",United States,"$7,000,000 (estimated)","$34,348,443",PG,English,"Twentieth Century Fox, Island World","Baseball, Coming-of-Age, Teen Comedy, Comedy, ...",David Mickey Evans,"David Mickey Evans, Robert Gunter"
498,https://www.imdb.com/title/tt28142896/?ref_=sr...,Weekend in Taipei,"Luke Evans, Sung Kang, Lun-Mei Gwei, Wyatt Yan...",1 hour 40 minutes,None,"September 25, 2024 (France)",France,None,None,R,English,"EuropaCorp, Rabbits Black","Action, Thriller",George Huang,"Luc Besson, George Huang"


In [22]:
# Ensure all lists are padded to the same length before creating the DataFrame
description = pad_list(description, expected_length)

# Create the DataFrame as usual
data = {
    'Links': movie_links,
    'Movie': movies,
    'Actors': film_actors,
    'Duration': duration,
    'Ratings': ratings,
    'Release Date': release_date,
    'Country': country,
    'Budget': budget,
    'Box Office': boxoffice,
    'Certificate': Certificate,
    'Original Language': original_language,
    'Companies': companies,
    'Genres': movie_category,
    'Directors': directors,
    'Writing Credits': writing_credits,
    'Description': description  # Now the lengths should match
}

df = pd.DataFrame(data)
df.to_csv('IMDB_Top_Movies.csv', index=False)

print("DataFrame successfully exported to IMDB_Top_Movies.csv")


DataFrame successfully exported to IMDB_Top_Movies.csv


In [24]:
import pandas as pd

# Read the CSV files into DataFrames
df1 = pd.read_csv('IMDB_Top_Movies.csv')
df2 = pd.read_csv('movies_data.csv')

# Concatenate the DataFrames
# If you want to concatenate them vertically (rows added)
merged_df = pd.concat([df1, df2], ignore_index=True)

# If you want to concatenate them horizontally (columns added)
# merged_df = pd.concat([df1, df2], axis=1)

# Save the concatenated DataFrame to a new CSV file
merged_df.to_csv('1000 IMDB.csv', index=False)

print("CSV files successfully merged into merged_file.csv")


CSV files successfully merged into merged_file.csv
